In [1]:
%set_env BASELINE_DIR=../../../example_designs
%set_env DESIGNS_DIR=../../../example_designs/example_designs/sparseloop
%set_env SCRIPTS_DIR=../scripts

env: BASELINE_DIR=../../../example_designs
env: DESIGNS_DIR=../../../example_designs/example_designs/sparseloop
env: SCRIPTS_DIR=../scripts


# Stage 0: NO sparse optimizations at all

In [2]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage0/ && cat problem.yaml


    name: system
    attributes:
      technology: "45nm"

In [3]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage0/ && cat arch.yaml

architecture:
  version: 0.4
  nodes:

  - !Container
    name: system
    attributes:
      technology: "45nm"
      global_cycle_seconds: 1e-9

  - !Component
    name: DRAM
    class: DRAM
    attributes:
      type: "LPDDR4"
      width: 64
      datawidth: 8
      depth: 524288

  - !Container
    name: eyeriss

  - !Component
    name: shared_glb
    class: smartbuffer_SRAM
    attributes:
      depth: 16384
      width: 64
      n_banks: 32
      datawidth: 8
      read_bandwidth: 16
      write_bandwidth: 16
    constraints:
      dataspace:
        keep: [Inputs, Outputs]
        bypass: [Weights]

  - !Container
    name: PE_column
    spatial: {meshX: 12}
    constraints:
      spatial:
        permutation: [P, Q, R]
        factors: [P=1, Q=1, R=1]
        split: 3

  - !Container
    name: PE
    spatial: {meshY: 14}
    constraints:
      spatial:
        permutation: [P, Q, R]
        factors: [P=1, Q=1, R=1]
        split: 3

  - !Parallel
    nodes:

    - !Component
 

In [10]:
%%bash
# cd $DESIGNS_DIR/eyeriss-like-matmul/ && cat mapping.yaml

In [5]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage0

input file: /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage0/outputs/default_problem/parsed-processed-input.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage0/outputs/default_problem/parsed-processed-input.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 4
Mapper configuration complete.
Initializing Index Fac

In [6]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage1/ && cat problem.yaml

problem:
  version: 0.4
  shape:
    name: Sparse_MatMul
    dimensions: [ P, Q, R ]
    data_spaces:
    - name: Weights
      projection:
        - [ [ P ] ]
        - [ [ Q ] ]
    - name: Inputs
      projection:
        - [ [ Q ] ]
        - [ [ R ] ]
    - name: Outputs
      projection:
        - [ [ P ] ]
        - [ [ R ] ]
      read_write: True

  instance:
    P: 256
    Q: 32
    R: 32
    densities:
      Inputs:
        distribution: fixed_structured
        density: 0.30
      Weights:
        distribution: fixed_structured
        density: 0.40


# Stage 1: Add Representation Format Only

In [7]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage1/ && cat arch.yaml

architecture:
  version: 0.4
  nodes:

  - !Container
    name: system
    attributes:
      technology: "45nm"
      global_cycle_seconds: 1e-9

  - !Component
    name: DRAM
    class: DRAM
    attributes:
      type: "LPDDR4"
      width: 64
      datawidth: 8
      depth: 524288
    sparse_optimizations:
      representation_format:
        data_spaces:
          - name: Inputs
            ranks:
              - format: UOP
              - format: CP
          - name: Weights
            ranks:
              - format: UOP
              - format: CP
          - name: Outputs
            ranks:
              - format: UOP
              - format: CP

  - !Container
    name: eyeriss

  - !Component
    name: shared_glb
    class: smartbuffer_SRAM
    attributes:
      depth: 16384
      width: 64
      n_banks: 32
      datawidth: 8
      read_bandwidth: 16
      write_bandwidth: 16
    constraints:
      dataspace: {keep: [Inputs, Outputs], bypass: [Weights]}
    sparse_optimizations

In [8]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage1

input file: /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage1/outputs/default_problem/parsed-processed-input.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage1/outputs/default_problem/parsed-processed-input.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 4
Mapper configuration complete.
Initializing Index Fac

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/common/backend_calls.py", line 186, in _parse_output
    return specification._parse_timeloop_output(
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/specification.py", line 117, in _parse_timeloop_output
    return parse_timeloop_output(self, timeloop_output_dir, prefix)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 465, in parse_timeloop_output
    parse_stats_file(stats_path)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 24, in parse_stats_file
    return parse_stats_stream(f)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 48, in parse_stats_stream
    assert cycles is not None, f"Could not find cycles in stats."
AssertionError: Could not find cycles in stats.

The above exception was the direct cause o

CalledProcessError: Command 'b'cd $BASELINE_DIR\npython3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage1\n'' returned non-zero exit status 1.

# Stage 2: Added Gating Only

In [9]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage2/ && cat problem.yaml

problem:
  version: 0.4
  shape:
    name: Sparse_MatMul
    dimensions: [ P, Q, R ]
    data_spaces:
    - name: Weights
      projection:
        - [ [ P ] ]
        - [ [ Q ] ]
    - name: Inputs
      projection:
        - [ [ Q ] ]
        - [ [ R ] ]
    - name: Outputs
      projection:
        - [ [ P ] ]
        - [ [ R ] ]
      read_write: True

  instance:
    P: 256
    Q: 32
    R: 32
    densities:
      Inputs:
        distribution: fixed_structured
        density: 0.30
      Weights:
        distribution: fixed_structured
        density: 0.40


In [10]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage2/ && cat arch.yaml

architecture:
  version: 0.4
  nodes:

  - !Container
    name: system
    attributes:
      technology: "45nm"
      global_cycle_seconds: 1e-9

  - !Component
    name: DRAM
    class: DRAM
    attributes:
      type: "LPDDR4"
      width: 64
      datawidth: 8
      depth: 524288
    sparse_optimizations:
      representation_format:
        data_spaces:
          - name: Inputs
            ranks:
              - format: UOP
              - format: CP
          - name: Weights
            ranks:
              - format: UOP
              - format: CP
          - name: Outputs
            ranks:
              - format: UOP
              - format: CP

  - !Container
    name: eyeriss

  - !Component
    name: shared_glb
    class: smartbuffer_SRAM
    attributes:
      depth: 16384
      width: 64
      n_banks: 32
      datawidth: 8
      read_bandwidth: 16
      write_bandwidth: 16
    constraints:
      dataspace: {keep: [Inputs, Outputs], bypass: [Weights]}
    sparse_optimizations

In [11]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage2

input file: /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage2/outputs/default_problem/parsed-processed-input.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage2/outputs/default_problem/parsed-processed-input.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 4
Mapper configuration complete.
Initializing Index Fac

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/common/backend_calls.py", line 186, in _parse_output
    return specification._parse_timeloop_output(
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/specification.py", line 117, in _parse_timeloop_output
    return parse_timeloop_output(self, timeloop_output_dir, prefix)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 465, in parse_timeloop_output
    parse_stats_file(stats_path)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 24, in parse_stats_file
    return parse_stats_stream(f)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 48, in parse_stats_stream
    assert cycles is not None, f"Could not find cycles in stats."
AssertionError: Could not find cycles in stats.

The above exception was the direct cause o

CalledProcessError: Command 'b'cd $BASELINE_DIR\npython3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage2\n'' returned non-zero exit status 1.

# Stage 3: Added Skip only

In [12]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage3/ && cat problem.yaml

problem:
  version: 0.4
  shape:
    name: Sparse_MatMul
    dimensions: [ P, Q, R ]
    data_spaces:
    - name: Weights
      projection:
        - [ [ P ] ]
        - [ [ Q ] ]
    - name: Inputs
      projection:
        - [ [ Q ] ]
        - [ [ R ] ]
    - name: Outputs
      projection:
        - [ [ P ] ]
        - [ [ R ] ]
      read_write: True

  instance:
    P: 256
    Q: 32
    R: 32
    densities:
      Inputs:
        distribution: fixed_structured
        density: 0.30
      Weights:
        distribution: fixed_structured
        density: 0.40


In [13]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage3/ && cat arch.yaml

architecture:
  version: 0.4
  nodes:

  - !Container
    name: system
    attributes:
      technology: "45nm"
      global_cycle_seconds: 1e-9

  - !Component
    name: DRAM
    class: DRAM
    attributes:
      type: "LPDDR4"
      width: 64
      datawidth: 8
      depth: 524288
    sparse_optimizations:
      representation_format:
        data_spaces:
          - name: Inputs
            ranks:
              - format: UOP
              - format: CP
          - name: Weights
            ranks:
              - format: UOP
              - format: CP
          - name: Outputs
            ranks:
              - format: UOP
              - format: CP

  - !Container
    name: eyeriss

  - !Component
    name: shared_glb
    class: smartbuffer_SRAM
    attributes:
      depth: 16384
      width: 64
      n_banks: 32
      datawidth: 8
      read_bandwidth: 16
      write_bandwidth: 16
    constraints:
      dataspace:
        keep: [Inputs, Outputs]
        bypass: [Weights]
    sparse_

In [14]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage3

input file: /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage3/outputs/default_problem/parsed-processed-input.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage3/outputs/default_problem/parsed-processed-input.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 4
Mapper configuration complete.
Initializing Index Fac

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/common/backend_calls.py", line 186, in _parse_output
    return specification._parse_timeloop_output(
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/specification.py", line 117, in _parse_timeloop_output
    return parse_timeloop_output(self, timeloop_output_dir, prefix)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 465, in parse_timeloop_output
    parse_stats_file(stats_path)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 24, in parse_stats_file
    return parse_stats_stream(f)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 48, in parse_stats_stream
    assert cycles is not None, f"Could not find cycles in stats."
AssertionError: Could not find cycles in stats.

The above exception was the direct cause o

CalledProcessError: Command 'b'cd $BASELINE_DIR\npython3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage3\n'' returned non-zero exit status 1.

# Stage 4: Adding everything

In [15]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage4/ && cat problem.yaml

problem:
  version: 0.4
  shape:
    name: Sparse_MatMul
    dimensions: [ P, Q, R ]
    data_spaces:
    - name: Weights
      projection:
        - [ [ P ] ]
        - [ [ Q ] ]
    - name: Inputs
      projection:
        - [ [ Q ] ]
        - [ [ R ] ]
    - name: Outputs
      projection:
        - [ [ P ] ]
        - [ [ R ] ]
      read_write: True

  instance:
    P: 256
    Q: 32
    R: 32
    densities:
      Inputs:
        distribution: fixed_structured
        density: 0.30
      Weights:
        distribution: fixed_structured
        density: 0.40


In [16]:
%%bash
cd $DESIGNS_DIR/eyeriss-like-matmul-stage4/ && cat arch.yaml

architecture:
  version: 0.4
  nodes:
  - !Container
    name: system
    attributes:
      technology: "45nm"
      global_cycle_seconds: 1e-9

  - !Component
    name: DRAM
    class: DRAM
    attributes:
      type: "LPDDR4"
      width: 64
      datawidth: 8
      depth: 524288
      metadata_storage_width: 16
      metadata_datawidth: 4
    sparse_optimizations:
      representation_format:
        data_spaces:
          - name: Inputs
            ranks:
              - format: UOP
              - format: CP
          - name: Weights
            ranks:
              - format: UOP
              - format: CP
          - name: Outputs
            ranks:
              - format: UOP
              - format: CP

  - !Container
    name: eyeriss

  - !Component
    name: shared_glb
    class: smartbuffer_SRAM
    attributes:
      depth: 16384
      width: 64
      n_banks: 32
      datawidth: 8
      read_bandwidth: 16
      write_bandwidth: 16
    constraints:
      dataspace: {keep: [I

In [17]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage4

input file: /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage4/outputs/default_problem/parsed-processed-input.yaml
  _______                __                
 /_  __(_)___ ___  ___  / /___  ____  ____ 
  / / / / __ `__ \/ _ \/ / __ \/ __ \/ __ \
 / / / / / / / / /  __/ / /_/ / /_/ / /_/ /
/_/ /_/_/ /_/ /_/\___/_/\____/\____/ .___/ 
                                  /_/      

Problem configuration complete.
execute:/usr/local/bin/accelergy /home/workspace/example_designs/example_designs/sparseloop/eyeriss-like-matmul-stage4/outputs/default_problem/parsed-processed-input.yaml --oprefix timeloop-mapper. -o ./ > timeloop-mapper.accelergy.log 2>&1
Generate Accelergy ERT (energy reference table) to replace internal energy model.
Generate Accelergy ART (area reference table) to replace internal area model.
Architecture configuration complete.
Sparse optimization configuration complete.
Using threads = 4
Mapper configuration complete.
Initializing Index Fac

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/common/backend_calls.py", line 186, in _parse_output
    return specification._parse_timeloop_output(
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/specification.py", line 117, in _parse_timeloop_output
    return parse_timeloop_output(self, timeloop_output_dir, prefix)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 465, in parse_timeloop_output
    parse_stats_file(stats_path)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 24, in parse_stats_file
    return parse_stats_stream(f)
  File "/usr/local/lib/python3.10/dist-packages/pytimeloop/timeloopfe/v4/output_parsing.py", line 48, in parse_stats_stream
    assert cycles is not None, f"Could not find cycles in stats."
AssertionError: Could not find cycles in stats.

The above exception was the direct cause o

CalledProcessError: Command 'b'cd $BASELINE_DIR\npython3 run_example_designs.py --architecture sparseloop/eyeriss-like-matmul-stage4\n'' returned non-zero exit status 1.